In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sparkNlp") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.4.5") \
    .getOrCreate()

In [3]:
from pyspark import SparkContext
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
from functools import reduce
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud 
import pandas as pd
import re
import string

In [4]:
cong_2017 = spark.read.csv("s3://502finalprojbucky/congresstweets/data/June2017.csv/*.part",header=True)

In [5]:
cong_2017.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- id: string (nullable = true)
 |-- screen_name: string (nullable = true)
 |-- time: string (nullable = true)
 |-- link: string (nullable = true)
 |-- text: string (nullable = true)
 |-- source: string (nullable = true)
 |-- user_id: string (nullable = true)



In [6]:
cong_2017_rdd = cong_2017.select("text").rdd.flatMap(lambda x: x)

In [7]:
cong_2017_rdd.take(5)

['Listen to these stories &amp; join us in the fight for more affordable, more accessible health care. http://pbs.twimg.com/media/DDWDa6DW0AAS3XZ.jpg',
 'RT @HomelandDems Ranking Member @BennieGThompson to @DHSgov Secretary Kelly: We must see clear guidance now on SCOTUS rules re: #TravelBan  implementation http://pbs.twimg.com/media/DDR24gYXsAAUwcK.jpg http://pbs.twimg.com/media/DDR25XvW0AAc1wB.jpg',
 'RT @LADeptHealth @RepRichmond @nwlc @AARPLouisiana @nationalcouncil 4. #SenateHealthCareBill substantially increases costs for LA private insurance shoppers - as much as 270%+ for older folks',
 "A victory for the American people, especially farmers and foresters in Southwest Alabama who would have suffered under this rule. #WOTUS https://twitter.com/EPAScottPruitt/status/879770792950730752 QT @EPAScottPruitt Today's #WOTUS action is a significant step to return power to states &amp; provide regulatory certainty to our nation’s farmers and businesses. http://pbs.twimg.com/media/DDWS3deU

In [8]:
lowerCase_cong_2017_rdd = cong_2017_rdd.map(lambda x : x.lower())

In [9]:
lowerCase_cong_2017_rdd.take(5)

['listen to these stories &amp; join us in the fight for more affordable, more accessible health care. http://pbs.twimg.com/media/ddwda6dw0aas3xz.jpg',
 'rt @homelanddems ranking member @benniegthompson to @dhsgov secretary kelly: we must see clear guidance now on scotus rules re: #travelban  implementation http://pbs.twimg.com/media/ddr24gyxsaauwck.jpg http://pbs.twimg.com/media/ddr25xvw0aac1wb.jpg',
 'rt @ladepthealth @reprichmond @nwlc @aarplouisiana @nationalcouncil 4. #senatehealthcarebill substantially increases costs for la private insurance shoppers - as much as 270%+ for older folks',
 "a victory for the american people, especially farmers and foresters in southwest alabama who would have suffered under this rule. #wotus https://twitter.com/epascottpruitt/status/879770792950730752 qt @epascottpruitt today's #wotus action is a significant step to return power to states &amp; provide regulatory certainty to our nation’s farmers and businesses. http://pbs.twimg.com/media/ddws3deu

In [10]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/hadoop/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
nltk.download('stopwords')
nltk.download('words')

[nltk_data] Downloading package stopwords to /home/hadoop/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /home/hadoop/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [12]:
def sent_TokenizeFunct(x):
    return nltk.sent_tokenize(x)

In [13]:
sentenceTokenizeRDD = lowerCase_cong_2017_rdd.map(sent_TokenizeFunct)

In [14]:
sentenceTokenizeRDD.take(5)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 5.0 failed 4 times, most recent failure: Lost task 0.3 in stage 5.0 (TID 2050, ip-172-31-85-68.ec2.internal, executor 4): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/mnt/yarn/usercache/hadoop/appcache/application_1586888319542_0002/container_1586888319542_0002_01_000006/pyspark.zip/pyspark/worker.py", line 364, in main
    func, profiler, deserializer, serializer = read_command(pickleSer, infile)
  File "/mnt/yarn/usercache/hadoop/appcache/application_1586888319542_0002/container_1586888319542_0002_01_000006/pyspark.zip/pyspark/worker.py", line 69, in read_command
    command = serializer._read_with_length(file)
  File "/mnt/yarn/usercache/hadoop/appcache/application_1586888319542_0002/container_1586888319542_0002_01_000006/pyspark.zip/pyspark/serializers.py", line

In [ ]:
def word_TokenizeFunct(x):
    splitted = [word for line in x for word in line.split()]
    return splitted
wordTokenizeRDD = sentenceTokenizeRDD.map(word_TokenizeFunct)

In [ ]:
wordTokenizeRDD.take(5)

In [ ]:
# get the list of stopwords from nltk
from nltk.corpus import stopwords

eng_stopwords = stopwords.words('english')
eng_stopwords.append('rt')
eng_stopwords.append('qt')
eng_stopwords.append('&amp')
eng_stopwords.append('amp')
eng_stopwords.append('+')
eng_stopwords.append('w')